In [ ]:
import subprocess
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import json

from datetime import timedelta
from pathlib import Path

%load_ext autoreload
%autoreload 2


# Disable logging of the program in the notebook
os.environ["LOGLEVEL"] = "CRITICAL"

logging.basicConfig()
logging.root.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('__main__')
logger.setLevel(os.environ.get("LOGLEVEL", logging.FATAL))

pybooklogger = logging.getLogger('pybook')
pybooklogger.setLevel(logging.DEBUG)

%aimport generation

from old_generation import generate
from parseRePEAT import *

# Plotting for the arrival time function
Top plot is the arrival time set out over the departure time

Bottom plot is the sum of the delay of the agents


In [ ]:
def plot_atf(segments, axs, eatfs, **kwargs):
    color = kwargs.get('color', None)
    label = kwargs.get('label', None)
    y_offset = kwargs.get('y_offset', 0)

    if 'expected_arrival_time' in kwargs:
        eat = kwargs['expected_arrival_time']
        axs[0].axhline(eat)

    line = None
    for (x0, x1, y0, y1) in segments:
        if x0 == "-inf" and x1 != "inf" and y1 != "inf":
            axs[0].hlines(float(y1) + y_offset, 0, float(x1), colors=color)
        line, = axs[0].plot([float(x0), float(x1)], [float(y0) + y_offset, float(y1) + y_offset], color=color)
    line.set_label(label) if line is not None else None

    plotted_intervals = []
    for path_eatf in eatfs.values():
        for (zeta, alpha, beta, delta, gammas) in path_eatf:
            min_gamma = 0
            max_gamma = 0
            for gamma_min, gamma_max, rt in gammas:
                # if gamma_max > gamma_min:
                #     raise ValueError(f"Max gamma > Min gamma, {gamma_max} > {gamma_min}")
                min_gamma += max(float(gamma_min) - float(rt), 0)
                max_gamma += max(float(gamma_max) - float(rt), 0)

            alpha = float(alpha)
            beta = float(beta)

            plotted_intervals.append((min(alpha, beta), beta, min_gamma, max_gamma))

            if alpha <= beta:
                # axs[1].plot([previous_beta, alpha], [min_gamma + y_offset, min_gamma + y_offset], color=color)
                axs[1].plot([alpha, beta], [min_gamma + y_offset, max_gamma + y_offset], color=color)
            # else:
            #     # axs[1].plot([previous_beta, beta - (gamma_diff)], [min_gamma + y_offset, min_gamma + y_offset], color=color)
            #     axs[1].plot([beta - (gamma_diff), beta], [min_gamma + y_offset, max_gamma + y_offset], color=color)
            # axs[1].plot([float(alpha), float(beta)], [min_gammas, max_gammas], color=color)
            previous_beta = beta

    previous_beta = 0
    plotted_intervals.sort(key=lambda x: x[0])

    for (alpha, beta, min_gamma, max_gamma) in plotted_intervals:
        axs[1].plot([previous_beta, alpha], [min_gamma + y_offset, min_gamma + y_offset], color=color)
        previous_beta = beta

            # pybooklogger.info(f"{alpha}, {beta}, {gammas}, {min_gammas} - {max_gammas}")

In [ ]:
def setup_plt(**kwargs):
    widths = [10]
    heights = [4, 1]
    fig, axs = plt.subplots(ncols=1, nrows=2, gridspec_kw={"height_ratios": heights, "width_ratios": widths})
    axs = axs.transpose()
    fig.set_figheight(7)
    fig.set_figwidth(15)

    axs[0].set_xlabel("Departure time (hh:mm:ss)")
    axs[1].set_xlabel("Departure time (hh:mm:ss)")
    axs[0].set_ylabel("Arrival time (hh:mm:ss)")
    axs[1].set_ylabel("Total delay of agents (s)")
    axs[0].set_title("Arrival Time Function")
    axs[0].set_xlim(left=kwargs.get("min_x", None), right=kwargs.get("max_x", None))
    axs[1].set_xlim(left=kwargs.get("min_x", None), right=kwargs.get("max_x", None))
    axs[0].set_ylim(bottom=kwargs.get("min_y", None), top=kwargs.get("max_y", None))
    axs[0].grid()
    axs[1].grid()

    axs[0].set_xticklabels([str(timedelta(seconds=xtick)) for xtick in axs[0].get_xticks()])
    axs[0].set_yticklabels([str(timedelta(seconds=ytick)) for ytick in axs[0].get_yticks()])
    axs[1].set_xticklabels([str(timedelta(seconds=xtick)) for xtick in axs[1].get_xticks()])

    return fig, axs

def plot_experiments(exps, save_path=None, **kwargs):
    fig, axs = setup_plt(**kwargs)
    for e in exps:
        if e.results is None:
            pybooklogger.info(f"No results found, skipping {e}")
            continue
        pybooklogger.info(f"Plotting {e}")
        e.plot(axs, **kwargs)

    axs[0].legend()
    if save_path:
        fig.savefig(f"figures/{save_path}")
    plt.show()

# Classes to setup for the experiment

## Agent
Represents a train, where the id is -1 for planning a new train, or the index of a train, starting at 1, of a train that is trying to replan in the scenarios file

In [ ]:
class Agent:
    def __init__(self, id, origin, destination, velocity, start_time, **kwargs):
        self.id = id
        self.origin = origin
        self.destination = destination
        self.velocity = velocity
        self.start_time = start_time
        for key, value in kwargs.items():
            setattr(self, key, value)

    def __str__(self):
        return str(self.__dict__)

## Layout
holds to track layout and generates the track graph and route graph

In [ ]:
class Layout:
    def __init__(self, layout):
        self.g, self.g_block, self.g_duration, self.g_block_duration = generate.time_graph_creation(layout)

    def station_to_block(self, station, direction=0):
        if station + "a" in self.g_block.stations:
            station = station + "a"
        if station in self.g_block.stations:
            if direction == "A":
                direction = 0
            if direction == "B":
                direction = 1
            return self.g_block.stations[station][direction]
        pybooklogger.error(f"Station {station} not found")
        return station

    def get_path_for_agent(self, move, current_train, velocity):
        from generation.interval_generation import construct_path
        from generation.signal_sections import convertMovesToBlock

        path = construct_path(self.g, move, current_agent=current_train, agent_velocity=velocity)
        moves_per_agent = {current_train: [path]}
        return convertMovesToBlock(moves_per_agent, self.g, current_train)

## Scenario
Holds the unsafe intervals for every agent in the scenario file

In [ ]:
class Scenario:
    def __init__(self, l: Layout, scen_file, agent_id):
        self.l = l
        self.agent_id = agent_id
        self.block_intervals, self.moves_per_agent, self.unsafe_computation_time, self.block_routes, self.t_moves_to_block = generate.time_scenario_creation(scen_file, self.l.g, self.l.g_block, agent_id)

    def get_flexibility(self, max_buffer_time, use_recovery_time):
        return generate.time_flexibility_creation(self.block_routes, self.block_intervals, max_buffer_time, use_recovery_time)

    def plot(self, agent_to_plot_route_of, buffer_times, recovery_times, plot_route_of_agent_to_plot_route_of=True):
        exclude_agent=-1
        if not plot_route_of_agent_to_plot_route_of:
            exclude_agent=agent_to_plot_route_of
        generate.plot_route(agent_to_plot_route_of, self.moves_per_agent, self.block_routes, self.block_intervals, self.l.g_block, buffer_times, recovery_times, exclude_agent=exclude_agent)

## Experiment
Calculates the safe intervals given the max buffer time and recovery time.

Also can call the FlexSIPP search in C++.

In [ ]:
class Experiment:
    def __init__(self, s: Scenario, agent: Agent, max_buffer_time, use_recovery_time, metadata):
        self.s = s
        self.agent = agent
        self.metadata= metadata

        self.buffer_times, self.recovery_times, self.time_flexibility_creation = s.get_flexibility(max_buffer_time, use_recovery_time)
        self.safe_block_intervals, self.safe_block_edges_intervals, self.atfs, self.indices_to_states, self.safe_computation_time = generate.time_interval_creation(self.s.block_intervals, self.s.l.g_block, self.buffer_times, self.recovery_times, self.agent.destination, agent.velocity)
        self.results = None

    def run_search(self, timeout):
        file = "output"
        generate.write_intervals_to_file(file, self.safe_block_intervals, self.atfs, self.indices_to_states)
        try:
            proc = subprocess.run(["../search/buildDir/atsipp.exe", "--start", self.agent.origin, "--goal", self.agent.destination, "--edgegraph", file, "--search", "repeat", "--startTime", str(self.agent.start_time)], timeout=timeout, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)

        except subprocess.TimeoutExpired:
            pybooklogger.error(f'Timeout for repeat ({timeout}s) expired')
            return
        if int(proc.returncode) == 0:
            repeat_output = str(proc.stdout).split("'")[1].rsplit("\\r\\n")
            pybooklogger.debug(f"repeat output: {repeat_output}")
            metadata, catf, paths, eatfs = parse_list_of_outputs(repeat_output)
            pybooklogger.info(f"eats: {eatfs}")
            pybooklogger.info(f"cats: {catf}")
            self.results = (metadata, catf, paths, eatfs)


    def plot(self, axs, **kwargs):
        plot_atf(self.results[1], axs, self.results[3], label=self.metadata["label"], color=self.metadata["color"], y_offset=self.metadata["offset"], **kwargs)

    def get_running_time(self):
        return {
            "unsafe interval generation": self.s.unsafe_computation_time,
            "safe interval generation": self.safe_computation_time,
            "bt and crt generation": self.time_flexibility_creation,
            "converting routes to blocks": self.s.t_moves_to_block,
            "track graph creation": self.s.l.g_duration,
            "routing graph creation": self.s.l.g_block_duration,
            "FlexSIPP search time": float(self.results[0]["Search time"]) / 1000.0,
            "Lookup time": float(self.results[0]["Lookup time"]) / 1000.0,
        }

    def get_complexity(self):
        return {
            "nodes generated": int(self.results[0]["Nodes generated"]),
            "nodes decreased": int(self.results[0]["Nodes decreased"]),
            "nodes expanded": int(self.results[0]["Nodes expanded"]),
        }

In [ ]:
def run_experiments(exps: list[Experiment], timeout):
    for e in exps:
        e.run_search(timeout)

## Experiment setup
Create a list of experiments that can be run from the settings

In [ ]:
def setup_experiment(scenario: Scenario, overwrite_settings):
    experiments = []
    for exp in overwrite_settings:
        set_default(exp)
        origin = exp["origin"]
        destination = exp["destination"]
        velocity = exp["velocity"]
        start_time = exp["start_time"]
        max_buffer_time = exp["max_buffer_time"]
        use_recovery_time = exp["use_recovery_time"]
        metadata = exp["metadata"]

        agent_id = scenario.agent_id
        pybooklogger.info(f"Setting up experiment {exp}")

        origin_signal = scenario.l.station_to_block(origin)
        destination_signal = scenario.l.station_to_block(destination)
        agent = Agent(agent_id, origin_signal, destination_signal, velocity, start_time)


        experiments.append(Experiment(scenario, agent, max_buffer_time, use_recovery_time, metadata))
    return experiments

## Default settings
Default settings for when not everything is specified

In [ ]:
default_settings = {
    "origin": "ASD|13a",
    "destination": "RTD|2",
    "velocity": 140/3.6,
    "max_buffer_time": 0,
    "start_time": 0,
    "use_recovery_time": False,
    "metadata": {
        "color": "Red",
        "label": "No flexibility",
        "offset": 0,
    }
}

def _set_default(setting: dict, default: dict):
    for key, value in default.items():
        if key not in setting:
            setting[key] = value
        elif isinstance(value, dict):
            _set_default(setting[key], value)

def set_default(setting):
    _set_default(setting, default_settings)

# Track Layout
Calculate the layout of the dutch railway system


In [ ]:
layout_file =   "../data/prorail/parsed/netherlands-schiphol.json"
layout = Layout(layout_file)

In [ ]:
edges_df = pd.DataFrame({"Outgoing routes": [len(n.outgoing) for n in layout.g_block.nodes.values() if len(n.outgoing) <= 25]})
hist = edges_df.hist(bins=25, )
plt.xlabel("Number of outgoing routes")
plt.ylabel("Number of occurrences")
plt.show()

# Agents

In [ ]:
scenario_file = "../data/prorail/scenarios/SHL-2025-06-30.json"
try:
    base_path = Path(__file__).parent
    file_path = (base_path / scenario_file).resolve()
    data = json.load(open(file_path))
except:
    data = json.load(open(scenario_file))
types = {x["name"]: x for x in data["types"]}
agents = []
for trainNumber, entry in enumerate(data["trains"]):
    trainNumber += 1
    move = entry["movements"][0]
    velocity = types[entry["trainUnitTypes"][0]]["speed"] / 3.6

    block_path = layout.get_path_for_agent(move, trainNumber, velocity)

    agent = Agent(trainNumber, move["startLocation"], move["endLocation"], velocity, move["startTime"],
                  endTime=move["endTime"],
                  startTimeHuman=str(timedelta(seconds=move["startTime"])),
                  endTimeHuman=str(timedelta(seconds=move["endTime"])),
                  blockPath=block_path,
                  trainNumber=entry["trainNumber"],
                  trainUnitTypes=entry["trainUnitTypes"],
                  stops=move["stops"]
    )
    agents.append(agent)

agent_df = pd.DataFrame([agent.__dict__ for agent in agents])
agent_df['blockPathLength'] = agent_df['blockPath'].map(len)

print(f"total blocks in path: {agent_df['blockPathLength'].sum()}")
print(f"total edges in graph: {edges_df['Outgoing routes'].sum()}")

agent_df

# Experiment Shl -> Rtd

Second experiment as described in the thesis, where we plan in an extra train from Schiphol to Rotterdam, not going over the HSL line but through Leiden and The Hague.


## Scenario
First load the scenario where we don't include agent 1 (used for plotting the path) that is the Eurostar train.

In [ ]:
scenario_file = "../data/prorail/scenarios/SHL-2025-06-30.json"
# Replan these agents, -1 is planning in a new agent
agent_id = 1
scenario = Scenario(layout, scenario_file, agent_id)

## Experiment
Define three experiments:
1. Classical @SIPP, no extra flexibility
2. FlexSIPP but not including the recovery time
3. FlexSIPP

In [ ]:
# Setup experiment
experiment_settings = [
    {
        "start_time": 1080,
        "origin": "SHL|6",
        "destination": "RTD|2",
        "metadata": {
            "offset": 2,
        }
    },
    {
        "start_time": 1080,
        "max_buffer_time": 500,
        "origin": "SHL|6",
        "destination": "RTD|2",
        "metadata": {
            "color": "Green",
            "label": "Buffer time",
            "offset": 1,
        }
    },{
        "start_time": 1080,
        "origin": "SHL|6",
        "destination": "RTD|2",
        "max_buffer_time": 500,
        "use_recovery_time": True,
        "metadata": {
            "color": "Blue",
            "label": "Recovery time",
        }
    }
]

logger.setLevel(logging.CRITICAL)
experiments = setup_experiment(scenario, experiment_settings)

### Blocking staircase diagram
Showing the route of Shl -> Rtd

In [ ]:
for exp in experiments:
    exp.s.plot(1, exp.buffer_times, exp.recovery_times, False)

In [ ]:
timeout = 600
run_experiments(experiments, timeout)

## Results
### ATF Plot

In [ ]:
pybooklogger.setLevel(logging.WARNING)
experiments[0].metadata = {'color': 'Red', 'label': 'No flexibility', 'offset': 15}
experiments[1].metadata = {'color': 'Green', 'label': 'Buffer time', 'offset': 0}
experiments[2].metadata = {'color': 'Blue', 'label': 'Recovery time', 'offset': -15}
experiments[0].results[1][1] = ('1672.78', '2750.88', '5377.17', '5377.17')

kwargs = {"min_x": 1080, "max_x": 3000, "min_y": 2500, "max_y": 7000}
plot_experiments(experiments, **kwargs)


experiments[0].metadata = {'color': 'Red', 'label': 'No flexibility', 'offset': 0}
experiments[1].metadata = {'color': 'Green', 'label': 'Buffer time', 'offset': 0}
experiments[2].metadata = {'color': 'Blue', 'label': 'Recovery time', 'offset': 0}

plot_experiments([experiments[0]], **kwargs)
plot_experiments([experiments[1]], **kwargs)
plot_experiments([experiments[2]], **kwargs)

### Time statistics

In [ ]:
def sum_cols(df1, cols, name):
    df2 = df1.drop(columns=cols)
    df2[name] = df1[cols].sum(axis=1)
    return df2

time_df = pd.DataFrame([exp.get_running_time() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

setup_cols = ["track graph creation", "routing graph creation"]
recompute_cols = ["unsafe interval generation", "safe interval generation", "bt and crt generation", "converting routes to blocks"]
search_cols = ["FlexSIPP search time"]

time_df = sum_cols(time_df, setup_cols, "Setup Time")
time_df = sum_cols(time_df, recompute_cols, "Recompute Time")
time_df = sum_cols(time_df, search_cols, "Search Time")

print(time_df.to_latex())

### Search Node Statistics

In [ ]:
nodes_df = pd.DataFrame([exp.get_complexity() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

print(nodes_df.to_latex())

### Output paths found

In [ ]:
for key, value in experiments[2].results[3].items():
    delayed_trains = {i: v for i,v in enumerate(value[0][4]) if float(v[0]) > 0}
    print(f"{key.replace('r-', '')}\nearliest departure: {int(min(float(value[0][1]), float(value[0][2])) / 60)}\ndepart before: {int(float(value[0][2]) / 60)}\narrive at: {int((float(value[0][1]) + float(value[0][3])) / 60)}\ndelays trains: {delayed_trains}")

### Path statistics

In [ ]:
for exp in experiments:
    print(f"Differend paths found for {exp.metadata['label']}: {sum(exp.results[2].values())}")

# Experiment Gvc -> Shl and further

Experiment to test the validity against a TAD

## Scenario

Replan agents of the series 3500o

In [ ]:
scenario_file = "../data/prorail/scenarios/SHL-2025-06-30-TAD.json"
series_3500o = agent_df.loc[(agent_df['trainNumber'].str.startswith("35", na=False)) & (agent_df['trainNumber'].astype(int) % 2 == 1)]

# This train would be at 2700 at GV|6, we will be replanning its path till ASDZ|2
agent = series_3500o.iloc[1]
series_3500o

In [ ]:
# Replan these agents, -1 is planning in a new agent
agent_id = agent['id']
scenario = Scenario(layout, scenario_file, agent_id)

## Experiment
Define
three
experiments:
1.
Classical @ SIPP, no
extra
flexibility
2.
FlexSIPP
but
not including
the
recovery
time
3.
FlexSIPP



In [ ]:
# Setup experiment
experiment_settings = [
    {
        "start_time": 2700.0,
        "origin": "GV|6",
        "destination": "ASDZ|2",
        "metadata": {
            "offset": 2,
        }
    },
    {
        "start_time": 2700.0,
        "origin": "GV|6",
        "destination": "ASDZ|2",
        "max_buffer_time": 900,
        "metadata": {
            "color": "Green",
            "label": "Buffer time",
            "offset": 1,
        }
    }, {
        "start_time": 2700.0,
        "origin": "GV|6",
        "destination": "ASDZ|2",
        "max_buffer_time": 900,
        "use_recovery_time": True,
        "metadata": {
            "color": "Blue",
            "label": "Recovery time",
        }
    }
]

logger.setLevel(logging.CRITICAL)
experiments = setup_experiment(scenario, experiment_settings)

### Blocking staircase diagram
Showing the route of GV -> ASDZ|2 and more even, TODO

In [ ]:
for exp in experiments:
    exp.s.plot(agent_id, exp.buffer_times, exp.recovery_times, False)

In [ ]:
timeout = 600
pybooklogger.setLevel(logging.DEBUG)
run_experiments(experiments, timeout)

## Results
### ATF Plot

In [ ]:
pybooklogger.setLevel(logging.WARNING)
experiments[0].metadata = {'color': 'Red', 'label': 'No flexibility', 'offset': 5}
experiments[1].metadata = {'color': 'Green', 'label': 'Buffer time', 'offset': 0}
experiments[2].metadata = {'color': 'Blue', 'label': 'Recovery time', 'offset': -5}

kwargs = {"min_x": 2700, "max_x": 3100, "min_y": 4250, "max_y": 5200, "expected_arrival_time": 4920}
plot_experiments(experiments, **kwargs)


experiments[0].metadata = {'color': 'Red', 'label': 'No flexibility', 'offset': 0}
experiments[1].metadata = {'color': 'Green', 'label': 'Buffer time', 'offset': 0}
experiments[2].metadata = {'color': 'Blue', 'label': 'Recovery time', 'offset': 0}

plot_experiments([experiments[0]], **kwargs)
plot_experiments([experiments[1]], **kwargs)
plot_experiments([experiments[2]], **kwargs)

### Time statistics

In [ ]:
def sum_cols(df1, cols, name):
    df2 = df1.drop(columns=cols)
    df2[name] = df1[cols].sum(axis=1)
    return df2

time_df = pd.DataFrame([exp.get_running_time() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

setup_cols = ["track graph creation", "routing graph creation"]
recompute_cols = ["unsafe interval generation", "safe interval generation", "bt and crt generation", "converting routes to blocks"]
search_cols = ["FlexSIPP search time"]

time_df = sum_cols(time_df, setup_cols, "Setup Time")
time_df = sum_cols(time_df, recompute_cols, "Recompute Time")
time_df = sum_cols(time_df, search_cols, "Search Time")
time_df

### Search Node Statistics

In [ ]:
nodes_df = pd.DataFrame([exp.get_complexity() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

nodes_df

### Output paths found

In [ ]:
for key, value in experiments[2].results[3].items():
    delayed_trains = {i: v for i,v in enumerate(value[0][4]) if float(v[0]) > 0}
    print(f"{key.replace('r-', '')}\nearliest departure: {int(min(float(value[0][1]), float(value[0][2])) / 60)}\ndepart before: {int(float(value[0][2]) / 60)}\narrive at: {int((float(value[0][1]) + float(value[0][3])) / 60)}\ndelays trains: {delayed_trains}")

### Path statistics

In [ ]:
for exp in experiments:
    print(f"Differend paths found for {exp.metadata['label']}: {sum(exp.results[2].values())}")

# Experiment Runtime
Take a route of an agent with many stops, and run from start to every stop as an experiment


In [ ]:
scenario_file = "../data/prorail/scenarios/RT/2025-07-04_1.json"

try:
    base_path = Path(__file__).parent
    file_path = (base_path / scenario_file).resolve()
    data = json.load(open(file_path))
except:
    data = json.load(open(scenario_file))
types = {x["name"]: x for x in data["types"]}
agents = []
pybooklogger.setLevel(logging.CRITICAL)
logger.setLevel(logging.CRITICAL)
for trainNumber, entry in enumerate(data["trains"]):
    trainNumber += 1
    move = entry["movements"][0]
    velocity = types[entry["trainUnitTypes"][0]]["speed"] / 3.6
    block_path = layout.get_path_for_agent(move, trainNumber, velocity)

    agent = Agent(trainNumber, move["startLocation"], move["endLocation"], velocity, move["startTime"],
                  endTime=move["endTime"],
                  startTimeHuman=str(timedelta(seconds=move["startTime"])),
                  endTimeHuman=str(timedelta(seconds=move["endTime"])),
                  blockPath=block_path,
                  trainNumber=entry["trainNumber"],
                  trainUnitTypes=entry["trainUnitTypes"],
                  stops=move["stops"]
    )
    agents.append(agent)

agent_df = pd.DataFrame([agent.__dict__ for agent in agents])
agent_df['blockPathLength'] = agent_df['blockPath'].map(len)

print(f"total blocks in path: {agent_df['blockPathLength'].sum()}")
print(f"total edges in graph: {edges_df['Outgoing routes'].sum()}")

agent_df

In [ ]:
longest = agent_df.loc[agent_df['stops'].map(len).idxmax()]

# This train would be at 2700 at GV|6, we will be replanning its path till ASDZ|2
longest


In [ ]:
agent_id = longest['id']
scenario = Scenario(layout, scenario_file, agent_id)

## Experiment
Define an experiment for every stop, and the original start and stop


In [ ]:
start_time = longest["start_time"]
origin = longest["origin"]

experiment_settings = [{
        "start_time": start_time,
        "origin": origin,
        "destination": longest["destination"],
        "metadata": {
            "expected_arrival": longest["endTime"],
            "label": f'route to {longest["destination"]}'
        }
}]

for stop in longest["stops"]:
    experiment_settings.append({
        "start_time": start_time,
        "origin": origin,
        "destination": stop["location"],
        "metadata": {
            "expected_arrival": stop["expected_arrival"],
            "label": f'route to {stop["location"]}'
        }
    })

logger.setLevel(logging.CRITICAL)
experiments = setup_experiment(scenario, experiment_settings)

### Blocking staircase diagram
Showing the route of the agent with the most stops, its quite long.


In [ ]:
exp = experiments[0]
exp.s.plot(agent_id, exp.buffer_times, exp.recovery_times, True)

In [ ]:
timeout = 600
pybooklogger.setLevel(logging.DEBUG)
run_experiments(experiments, timeout)

## Results


### Time statistics

In [ ]:
def sum_cols(df1, cols, name):
    df2 = df1.drop(columns=cols)
    df2[name] = df1[cols].sum(axis=1)
    return df2

time_df = pd.DataFrame([exp.get_running_time() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

setup_cols = ["track graph creation", "routing graph creation"]
recompute_cols = ["unsafe interval generation", "safe interval generation", "bt and crt generation", "converting routes to blocks"]
search_cols = ["FlexSIPP search time"]

time_df = sum_cols(time_df, setup_cols, "Setup Time")
time_df = sum_cols(time_df, recompute_cols, "Recompute Time")
time_df = sum_cols(time_df, search_cols, "Search Time")
time_df

### Search Node Statistics


In [ ]:
nodes_df = pd.DataFrame([exp.get_complexity() for exp in experiments], index=[exp.metadata['label'] for exp in experiments])

nodes_df

### Output paths found


In [ ]:
for key, value in experiments[2].results[3].items():
    delayed_trains = {i: v for i,v in enumerate(value[0][4]) if float(v[0]) > 0}
    print(f"{key.replace('r-', '')}\nearliest departure: {int(min(float(value[0][1]), float(value[0][2])) / 60)}\ndepart before: {int(float(value[0][2]) / 60)}\narrive at: {int((float(value[0][1]) + float(value[0][3])) / 60)}\ndelays trains: {delayed_trains}")

### Path statistics


In [ ]:
for exp in experiments:
    print(f"Differend paths found for {exp.metadata['label']}: {sum(exp.results[2].values())}")